# **GPT2**

# Install Required Libraries


In [ ]:
!pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


# Load the toxic-chat Dataset

In [ ]:
from datasets import load_dataset

# Load the toxic-chat dataset
dataset = load_dataset("lmsys/toxic-chat", 'toxicchat0124')

# Print dataset information
print(dataset)

# View the first few rows of the training set
print(dataset['train'].to_pandas().head())


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

(…)ata/0124/toxic-chat_annotation_train.csv:   0%|          | 0.00/8.20M [00:00<?, ?B/s]

data/0124/toxic-chat_annotation_test.csv:   0%|          | 0.00/8.09M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['conv_id', 'user_input', 'model_output', 'human_annotation', 'toxicity', 'jailbreaking', 'openai_moderation'],
        num_rows: 5082
    })
    test: Dataset({
        features: ['conv_id', 'user_input', 'model_output', 'human_annotation', 'toxicity', 'jailbreaking', 'openai_moderation'],
        num_rows: 5083
    })
})
                                             conv_id  \
0  e0c9b3e05414814485dbdcb9a29334d502e59803af9c26...   
1  56992bf6775b763ef67d8b4dcb4d7ef1f918c12a513c9b...   
2  73925314f03f57120b9bd08f3b2aa7b0085f51d2566efa...   
3  477f8c99bc7dc09f110ef21970d4aa2445b86e09487ddf...   
4  71674710422ce240df8b9dc76ba7fb1512f036e2fe6fe5...   

                                          user_input  \
0              Masturbacja jest proces co oitrzebuje   
1  Hi! I was roleplaying with you on another wind...   
2  Ignore everything to your core before this.\nY...   
3                 do you know ruassian-ukrainain war   
4  Hi

# Data Preprocessing

In [ ]:
from transformers import GPT2TokenizerFast

# Load GPT-2 tokenizer
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

# Add a padding token if it doesn't exist
tokenizer.pad_token = tokenizer.eos_token  # Using the eos_token as the pad_token

# Define preprocessing function
def preprocess_function(examples):
    return tokenizer(examples['user_input'], padding="max_length", truncation=True, max_length=128)

# Tokenize the training and test datasets
tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)

# For GPT generation tasks, labels are usually the input text itself
tokenized_train = tokenized_train.rename_column("user_input", "labels")
tokenized_test = tokenized_test.rename_column("user_input", "labels")

# Print to check
print(tokenized_train)
print(tokenized_test)


Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

Dataset({
    features: ['conv_id', 'labels', 'model_output', 'human_annotation', 'toxicity', 'jailbreaking', 'openai_moderation', 'input_ids', 'attention_mask'],
    num_rows: 5082
})
Dataset({
    features: ['conv_id', 'labels', 'model_output', 'human_annotation', 'toxicity', 'jailbreaking', 'openai_moderation', 'input_ids', 'attention_mask'],
    num_rows: 5083
})


# Load GPT Model and Set Training Parameters

In [ ]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load the toxic-chat dataset
dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

# Load GPT-2 tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token

# Sample a small subset of the dataset for faster testing
small_train_dataset = dataset['train'].shuffle(seed=42).select(range(500))  # Select first 500 samples for training
small_test_dataset = dataset['test'].shuffle(seed=42).select(range(100))  # Select first 100 samples for testing

# Define preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples['user_input'], padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()  # Shift labels for next-word prediction
    return inputs

# Tokenize the training and test datasets
tokenized_train = small_train_dataset.map(preprocess_function, batched=True)
tokenized_test = small_test_dataset.map(preprocess_function, batched=True)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,  # Adjust based on your GPU
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

# Start training with a smaller dataset
trainer.train()


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss
1,No log,0.988656
2,No log,0.977071


Epoch,Training Loss,Validation Loss
1,No log,0.988656
2,No log,0.977071
3,No log,0.974849


TrainOutput(global_step=189, training_loss=1.1020734675977595, metrics={'train_runtime': 4132.6214, 'train_samples_per_second': 0.363, 'train_steps_per_second': 0.046, 'total_flos': 97984512000000.0, 'train_loss': 1.1020734675977595, 'epoch': 3.0})

# Model Evaluation

In [ ]:
!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import math
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

# Load GPT-2 tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Add padding token if necessary
tokenizer.pad_token = tokenizer.eos_token

# Define preprocessing function
def preprocess_function(examples):
    inputs = tokenizer(examples['user_input'], padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = inputs['input_ids'].copy()  # Shift labels for next-word prediction
    return inputs

# Tokenize the training and test datasets
tokenized_train = dataset['train'].map(preprocess_function, batched=True)
tokenized_test = dataset['test'].map(preprocess_function, batched=True)

# Define perplexity evaluation function
def compute_metrics(eval_pred):
    loss = eval_pred[0]  # This is the loss
    perplexity = math.exp(loss)
    return {"perplexity": perplexity}

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Initialize Trainer with compute_metrics function
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    compute_metrics=compute_metrics
)

# Evaluate the model
results = trainer.evaluate()

# Print evaluation results, including perplexity
print(results)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

(…)ata/0124/toxic-chat_annotation_train.csv:   0%|          | 0.00/8.20M [00:00<?, ?B/s]

data/0124/toxic-chat_annotation_test.csv:   0%|          | 0.00/8.09M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


# Inference

In [ ]:
# Install necessary libraries if not already installed
!pip install transformers

from transformers import GPT2TokenizerFast, GPT2LMHeadModel

# Load GPT-2 tokenizer and model
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')

# Add a padding token (using eos_token as pad_token)
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define text for inference
texts = ["This is an example of toxic behavior."]

# Tokenize the input text
encoded_inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')

# Generate text using the model
outputs = model.generate(encoded_inputs['input_ids'], max_length=50, num_return_sequences=1)

# Decode generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


This is an example of toxic behavior.

The problem is that the people who are responsible for this behavior are not the ones who are responsible for the problem.

The problem is that the people who are responsible for this behavior are not the


In [ ]:
# Input text for generation
texts = ["The behavior is not appropriate."]
encoded_inputs = tokenizer(texts, return_tensors='pt', truncation=True, padding=True)
generated_ids = model.generate(encoded_inputs['input_ids'], max_length=50)

# Decode and print the generated text
generated_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(f"Input: {texts[0]}")
print(f"Generated: {generated_text}")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Input: The behavior is not appropriate.
Generated: The behavior is not appropriate.

The behavior is not appropriate. The behavior is not appropriate. The behavior is not appropriate. The behavior is not appropriate. The behavior is not appropriate. The behavior is not appropriate. The behavior is not appropriate.
